In [ ]:
import os
import numpy as np
from src.jobs import Train

repo_dir = '/home/cam/Documents/Enceladus/'
os.chdir(repo_dir)

args = dict(
    seed=np.random.seed(69),
    records_dir='/media/cam/CAPSTONEDB/database_tools/data-2022-10-17/mimic3/',
    out_dir='output/',
    model_dir='model/',
    use_multiprocessing=True,
    val_steps=None,
    steps_per_epoch=None,
    test_steps=None,
    use_wandb_tracking=False,
    wandb_entity=None,
    wandb_project=None,
    epochs=100,
    batch_size=32,
    learning_rate=0.0001,
    es_patience=5,
)

Train(args=args).run()